In [ ]:
# Logger
import logging
logging.basicConfig(level=logging.WARN)

# Enable automatic module reload
%load_ext autoreload
%autoreload 2

# Load PyALCS ACS2 module
from bacs.agents.bacs import BACS, Configuration
from bacs.agents.bacs.utils.GymMazeWrapper import _maze_metrics, parse_metrics_to_df, plot_performance

# Load environments
import gym
import my_mazes

# Allow to parallelize all benchmarks to do
import parmap

# For calculation standard deviation
import statistics

# Agent - ACS2 - BENCHMARKING

## Main Parameters

In [ ]:
CLASSIFIER_LENGTH = 8
NUMBER_OF_POSSIBLE_ACTIONS = 8

NUMBER_OF_EXPLORE_TRIALS = 1000
DO_ACTION_PLANNING_EXPLORE = False
DO_GA_EXPLORE = False
DO_SUBSUMPTION_EXPLORE = True
BETA_EXPLORE = 0.05
EPSILON = 0.8

NUMBER_OF_EXPLOIT_TRIALS = 500
DO_ACTION_PLANNING_EXPLOIT = False
DO_SUBSUMPTION_EXPLOIT = True
BETA_EXPLOIT = 0.00

LENGTH_OF_BEHAVIORAL_SEQUENCES = 0

NUMBER_OF_ITERATIONS_TO_BENCH = 30

NB_OF_PROCESSES = 24

## Full Details of Agent Parameters

In [ ]:
cfg_explore = Configuration(
    classifier_length=CLASSIFIER_LENGTH,
    number_of_possible_actions=NUMBER_OF_POSSIBLE_ACTIONS,
    user_metrics_collector_fcn=_maze_metrics,
    metrics_trial_frequency=1,
    do_ga=DO_GA_EXPLORE,
    do_subsumption=DO_SUBSUMPTION_EXPLORE,
    do_action_planning=DO_ACTION_PLANNING_EXPLORE,
    action_planning_frequency=50,
    beta=BETA_EXPLORE,
    gamma=0.95,
    theta_i=0.1,
    theta_r=0.9,
    epsilon=EPSILON,
    u_max=CLASSIFIER_LENGTH,
    theta_exp=20,
    theta_ga=100,
    theta_as=20,
    mu=0.3,
    chi=0.8,
    bs_max=LENGTH_OF_BEHAVIORAL_SEQUENCES
)

cfg_exploit = Configuration(
    classifier_length=CLASSIFIER_LENGTH,
    number_of_possible_actions=NUMBER_OF_POSSIBLE_ACTIONS,
    user_metrics_collector_fcn=_maze_metrics,
    metrics_trial_frequency=1,
    do_ga=False,
    do_subsumption=DO_SUBSUMPTION_EXPLOIT,
    do_action_planning=DO_ACTION_PLANNING_EXPLOIT,
    action_planning_frequency=50,
    beta=BETA_EXPLOIT,
    gamma=0.95,
    theta_i=0.1,
    theta_r=0.9,
    epsilon=0.0,
    u_max=CLASSIFIER_LENGTH,
    theta_exp=20,
    theta_ga=100,
    theta_as=20,
    mu=0.3,
    chi=0.8,
    bs_max=LENGTH_OF_BEHAVIORAL_SEQUENCES
)

### Benchmarking - Maze

In [ ]:
def bench_on_maze(env,n):
    avg_explore_list = []
    avg_exploit_list = []
    knowledge_list = []
    population_list = []
    numerosity_list = []
    reliable_list = []
    
    for i in range(n):
        
        # Initialize environment
        maze = gym.make(env.id)
        
        # Reset it, by putting an agent into random position
        situation = maze.reset()
    
        # Training of ACS2 - Exploration
        agent_explore = BACS(cfg_explore)
        population_explore, metrics_explore = agent_explore.explore(maze, NUMBER_OF_EXPLORE_TRIALS)
    
        # Using ACS2 - Exploitation
        agent_exploit = BACS(cfg_exploit, population_explore)
        population_exploit, metrics_exploit = agent_exploit.exploit(maze, NUMBER_OF_EXPLOIT_TRIALS)
        
        # Get average in exploration
        avg_step_explore = 0
        for trial in metrics_explore:
            avg_step_explore += trial['steps_in_trial']
        avg_step_explore /= NUMBER_OF_EXPLORE_TRIALS
        avg_explore_list.append(avg_step_explore)
        
        # Get average in exploitation
        avg_step_exploit = 0
        for trial in metrics_exploit:
            avg_step_exploit += trial['steps_in_trial']
        avg_step_exploit /= NUMBER_OF_EXPLOIT_TRIALS
        avg_exploit_list.append(avg_step_exploit)
        
        # Get other metrics
        knowledge_list.append(metrics_exploit[-1]['knowledge'])
        population_list.append(metrics_exploit[-1]['population'])
        numerosity_list.append(metrics_exploit[-1]['numerosity'])
        reliable_list.append(metrics_exploit[-1]['reliable'])
        
        print(env.id,'|','#' * (i+1) ,' ' * (n-(i+1)), '|')
    
    # Compute the means and standard deviations
    avg_explore = float(sum(avg_explore_list)) / n
    std_explore = statistics.pstdev(avg_explore_list)
    avg_exploit = float(sum(avg_exploit_list)) / n
    std_exploit = statistics.pstdev(avg_exploit_list)
    avg_knowledge = float(sum(knowledge_list)) / n
    std_knowledge = statistics.pstdev(knowledge_list)
    avg_population = float(sum(population_list)) / n
    std_population = statistics.pstdev(population_list)
    avg_numerosity = float(sum(numerosity_list)) / n
    std_numerosity = statistics.pstdev(numerosity_list)
    avg_reliable = float(sum(reliable_list)) / n
    std_reliable = statistics.pstdev(reliable_list)
    
    result = {
        'maze'             : env.id,
        'avg_explore'      : avg_explore,
        'std_explore'      : std_explore,
        'avg_exploit'      : avg_exploit,
        'std_exploit'      : std_exploit,
        'avg_knowledge'    : avg_knowledge,
        'std_knowledge'    : std_knowledge,
        'avg_population'   : avg_population,
        'std_population'   : std_population,
        'avg_numerosity'   : avg_numerosity,
        'std_numerosity'   : std_numerosity,
        'avg_reliable'     : avg_reliable,
        'std_reliable'     : std_reliable,
        'avg_exploit_list' : avg_exploit_list
    }
    
    print(result)
    
    return result

Custom functions for getting available environments in Gym depending on the type of mazes :

In [ ]:
filter_envs_typeIII = lambda env: env.id.startswith("Maze10-") or env.id.startswith("MazeE1") \
    or env.id.startswith("MazeE2") or env.id.startswith("Woods10")

filter_envs_typeII = lambda env: env.id.startswith("MazeF4") or env.id.startswith("Maze7") \
    or env.id.startswith("MiyazakiB")

filter_envs_typeI = lambda env: env.id.startswith("MazeB") or env.id.startswith("MazeD") \
    or env.id.startswith("Littman") or env.id.startswith("MiyazakiA") \
    or env.id.startswith("Cassandra")

filter_envs_na = lambda env: env.id.startswith("MazeF1") or env.id.startswith("MazeF2") \
    or env.id.startswith("MazeF3") or env.id.startswith("Woods1-") \
    or env.id.startswith("Woods14") or env.id.startswith("Maze4") \
    or env.id.startswith("Maze5") or env.id.startswith("MazeA")

all_envs = [env for env in gym.envs.registry.all()]

Benchmarking without the genetic algorithms on mazes :

In [ ]:
maze_envs = [env for env in all_envs if filter_envs_na(env) or filter_envs_typeI(env) or filter_envs_typeII(env) or filter_envs_typeIII(env)]

results = parmap.map(bench_on_maze, maze_envs, NUMBER_OF_ITERATIONS_TO_BENCH, pm_pbar=True, pm_processes=NB_OF_PROCESSES)

Parsing the previous result to get a markdown string for each environment :

In [ ]:
markdown_str = ''
for item in results:
    markdown_str += '|' + item['maze'] + '|'
    markdown_str += "{:.2f}".format(item['avg_explore']) + '|'
    markdown_str += "{:.2f}".format(item['std_explore']) + '|'
    markdown_str += "{:.2f}".format(item['avg_exploit']) + '|'
    markdown_str += "{:.2f}".format(item['std_exploit']) + '|'
    markdown_str += "{:.2f}".format(min(item['avg_exploit_list'])) + '|'
    markdown_str += "{:.2f}".format(max(item['avg_exploit_list'])) + '|'
    markdown_str += '0' + '|'
    markdown_str += "{:.2f}".format(item['avg_knowledge']) + '|'
    markdown_str += "{:.2f}".format(item['std_knowledge']) + '|'
    markdown_str += "{:.2f}".format(item['avg_population']) + '|'
    markdown_str += "{:.2f}".format(item['std_population']) + '|'
    markdown_str += "{:.2f}".format(item['avg_reliable']) + '|'
    markdown_str += "{:.2f}".format(item['std_reliable']) + '|'
    markdown_str += '\n'
print(markdown_str)

### Last results
 
The parameters used are:
 
- no use of action planning
- use of subsumption in the anticipatory learning process
- use od subsumption in the genetic algorithm is this last one is used
- $\gamma$ = 0.95
- $\theta_i$ = 0.1
- $\theta_r$ = 0.9
- $u_{max}$ = 8 (length of the classifier)
- $\theta_{exp}$ = 20
- $\beta$ = 0.05
- $\epsilon$ = 0.8
 
If genetic algorithms are used:

- $\theta_{ga}$ = 100
- $\theta_{as}$ = 20
- $\mu$ = 0.3
- $\chi$ = 0.8

The mazes in the following table have been sorted by aliasing type and then by complexity top-down.

For 1000 trials in Exploration and 500 trials in Exploitation for all environments, computed 30 times for calculating all averages, with a random function that choose a random action when exploring :
 
 
| | Exploration Avg | Exploration Std | Exploration Avg | Exploration Std | Best Exploration | Worst Exploration | Successful tries | Knowledge Avg | Knowledge Std | Population Avg | Population Std | Reliable Avg | Reliable Std |
|------------|:-----:|:-----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
|**Type III**     ||||||||||||||
|MazeE2-v0|28.16|1.12|59.72|3.82|52.56|70.67|0|37.87|1.98|668.07|56.80|378.20|44.29|
|Woods101demi-v0|35.57|1.88|68.34|3.23|62.73|74.77|0|52.38|0.00|109.50|10.12|72.93|9.58|
|Maze10-v0|51.81|1.51|74.34|9.17|53.62|95.84|0|65.46|2.37|119.67|12.82|96.63|11.88|
|Woods102-v0|38.08|1.27|57.76|8.80|44.89|69.17|0|51.26|0.22|152.23|17.40|116.03|17.44|
|Woods100-v0|16.16|0.48|34.31|2.12|28.64|39.24|0|60.00|0.00|18.07|0.36|14.07|0.36|
|Woods101-v0|31.34|1.32|41.79|2.55|37.01|47.74|0|63.09|0.66|96.43|10.10|83.00|9.71|
|MazeE1-v0|20.26|0.64|35.66|8.66|11.06|50.33|0|53.43|0.21|498.73|50.86|299.70|38.27|
|**Type II**      ||||||||||||||
|MazeF4-v0|56.35|2.35|53.31|6.68|42.03|74.98|0|88.13|2.83|100.90|7.44|89.47|8.23|
|Maze7-v0|49.65|2.08|47.34|7.75|24.77|59.06|0|83.02|2.93|93.23|7.84|86.63|7.98|
|MiyazakiB-v0|41.97|1.71|6.97|7.39|3.52|25.55|0|78.60|0.13|340.57|29.77|279.90|28.81|
|**Type I**       ||||||||||||||
|MazeB-v0|46.66|2.55|10.30|16.66|7.00|100.00|0|80.46|0.24|281.97|31.42|239.93|26.48|
|Littman89-v0|32.98|4.84|25.48|26.98|4.54|67.82|0|71.52|0.32|198.53|24.71|168.27|22.73|
|MiyazakiA-v0|27.12|1.62|8.42|11.53|3.34|43.81|0|70.71|0.17|332.27|31.20|250.43|24.45|
|MazeD-v0|34.64|1.61|4.00|0.00|4.00|4.00|0|82.08|2.06|245.03|37.94|209.37|33.41|
|Cassandra4x4-v0|13.72|0.66|4.94|9.28|2.36|54.86|0|51.03|0.92|247.20|29.37|145.67|19.23|
|Littman57-v0|22.06|1.69|43.83|24.26|3.53|65.14|0|72.52|2.95|100.07|8.00|92.43|8.02|
|**Not Aliased**||||||||||||||
|Maze4-v0|31.69|1.37|3.52|0.05|3.42|3.60|0|100.00|0.00|276.30|30.72|276.30|30.72|
|Maze5-v0|47.24|1.66|4.64|0.10|4.37|4.91|0|100.00|0.00|373.20|37.52|373.20|37.52|
|MazeA-v0|61.27|1.45|6.03|0.18|6.00|7.00|0|100.00|0.00|191.43|30.85|191.43|30.85|
|MazeF1-v0|10.05|0.28|1.80|0.04|1.70|1.87|0|100.00|0.00|79.50|6.37|79.50|6.37|
|MazeF2-v0|17.64|0.56|2.51|0.04|2.42|2.59|0|100.00|0.00|80.93|9.28|80.93|9.28|
|MazeF3-v0|26.71|0.70|3.38|0.04|3.29|3.47|0|100.00|0.00|93.43|10.14|93.43|10.14|
|Woods1-v0|9.01|0.25|1.62|0.03|1.55|1.67|0|100.00|0.00|121.57|23.39|121.57|23.39|
|Woods14-v0|68.74|1.59|9.49|0.17|9.19|9.85|0|100.00|0.00|121.50|15.39|121.50|15.39|

For 1000 trials in Exploration and 500 trials in Exploitation for all environments, computed 30 times for calculating all averages, with a random function that choose a random classifier when exploring :
 
 
| | Exploration Avg | Exploration Std | Exploration Avg | Exploration Std | Best Exploration | Worst Exploration | Successful tries | Knowledge Avg | Knowledge Std | Population Avg | Population Std | Reliable Avg | Reliable Std |
|------------|:-----:|:-----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
|**Type III**     ||||||||||||||
|MazeE2-v0|27.90|1.09|60.32|4.65|54.69|76.60|0|37.58|2.32|646.70|63.57|384.23|43.53|
|Woods101demi-v0|36.64|1.89|68.05|3.22|62.32|75.00|0|52.46|0.43|110.80|10.02|74.77|8.55|
|Maze10-v0|51.96|1.36|80.17|6.63|69.53|95.25|0|65.82|2.33|118.47|11.13|94.20|9.88|
|Woods102-v0|39.57|1.31|58.57|9.00|43.70|68.61|0|51.26|0.22|155.60|18.75|118.77|18.28|
|Woods100-v0|15.81|0.51|34.45|1.76|31.62|38.52|0|60.00|0.00|18.13|0.50|14.13|0.50|
|Woods101-v0|33.10|1.25|41.75|2.21|35.48|45.59|0|63.46|1.58|100.80|10.77|88.00|10.28|
|MazeE1-v0|21.02|1.00|37.70|8.21|22.05|55.83|0|53.26|0.36|486.37|44.55|293.07|29.78|
|**Type II**      ||||||||||||||
|MazeF4-v0|58.35|2.09|52.95|10.00|21.64|88.71|0|86.40|2.85|107.87|10.76|92.77|10.03|
|Maze7-v0|50.06|2.35|46.73|7.21|27.37|58.44|0|82.86|2.91|97.43|9.56|90.37|8.40|
|MiyazakiB-v0|41.20|2.05|4.23|2.81|3.51|19.33|0|78.57|0.00|342.43|26.51|280.87|25.01|
|**Type I**       ||||||||||||||
|MazeB-v0|35.66|2.12|9.76|12.70|3.87|43.87|0|80.49|0.29|281.50|26.29|237.70|23.43|
|Littman89-v0|33.53|3.80|26.78|27.28|4.62|66.66|0|71.52|0.32|202.13|27.71|171.03|25.66|
|MiyazakiA-v0|27.58|2.05|10.38|12.74|3.33|43.83|0|70.73|0.20|338.00|26.75|254.90|22.19|
|MazeD-v0|24.51|1.30|2.78|0.06|2.66|2.88|0|87.57|0.26|238.53|33.97|217.33|31.42|
|Cassandra4x4-v0|14.15|0.74|4.81|9.09|2.32|53.68|0|50.99|0.85|233.70|34.67|138.07|23.58|
|Littman57-v0|21.95|1.18|36.06|26.46|3.56|60.77|0|72.68|2.70|102.60|9.34|95.07|9.31|
|**Not Aliased**||||||||||||||
|Maze4-v0|32.10|1.45|3.51|0.06|3.40|3.66|0|100.00|0.00|296.77|37.49|296.77|37.49|
|Maze5-v0|46.37|2.91|4.65|0.07|4.50|4.82|0|100.00|0.00|367.07|34.97|367.07|34.97|
|MazeA-v0|48.51|1.61|4.32|0.24|4.07|5.32|0|100.00|0.00|183.90|25.45|183.90|25.45|
|MazeF1-v0|11.37|0.41|1.79|0.03|1.72|1.85|0|100.00|0.00|83.73|5.64|83.73|5.64|
|MazeF2-v0|19.80|0.88|2.51|0.03|2.44|2.57|0|100.00|0.00|83.33|9.91|83.33|9.91|
|MazeF3-v0|29.53|0.94|3.38|0.06|3.22|3.50|0|100.00|0.00|95.00|10.01|95.00|10.01|
|Woods1-v0|9.51|0.29|1.63|0.02|1.58|1.68|0|100.00|0.00|120.77|21.63|120.77|21.63|
|Woods14-v0|70.44|2.04|9.52|0.18|9.00|9.79|0|100.00|0.00|119.10|17.87|119.10|17.87|


### Complexity and Optimal Performance by Maze

These measures come from Zatuchna and Bagnall paper :

> Zhanna V. Zatuchna and Anthony Bagnall. 2009. Learning Mazes with Aliasing States: An LCS Algorithm with Associative Perception. Adaptive Behavior - Animals, Animats, Software Agents, Robots, Adaptive Systems 17, 1 (February 2009), 28-57. DOI=http://dx.doi.org/10.1177/1059712308099230

$\phi$ is about the average distance to reward in maze. Its calculation depends mainly on the type of maze. Here is reported $\phi$' if the maze is aliased, the original $\phi$ otherwise.

$\psi$ is about the complexity of the maze that depends on the average distance to reward and on the average steps taken by trained Q-learning.

I also added $d$, the maximal distance to food I consider to count the number of succesfull tries.

The question marks (**?**) highlight some discrepancies present in the original paper.

The asterisks (**\***) indicate pieces of information were not provided.

For further details, please see the original paper.

#### For Type III Aliased Mazes

|       |MazeE2      |Woods101demi|Maze10      |Woods102    |Woods100    |Woods101    |MazeE1       |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|:-----------:|
|$\phi$'|2.33        |3.1         |5.17        |3.31        |2.33        |2.9         |3.07         |
|$\psi$ |251.2       |251 **?**   |171         |167         |166         |149         |167 **?**    |

#### For Type II Aliased Mazes

|       |MazeF4      |Maze7       |Sutton      |Lab1        |MiyazakiB   |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|
|$\phi$'|4.5 **?**   |4.33        | **\***     | **\***     |3.33        |
|$\psi$ |47 **?**    |82 **?**    | **?**      | **?**      |1.03        |

#### For Type I Aliased Mazes

|       |MazeB       |Littman89   |MiyazakiA   |MazeD       |Cassandra4x4|Littman57   |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|
|$\phi$'|3.5         |3.77        |3.05        |2.75        |2.27        |3.71        |
|$\psi$ |1.26        |61 **?**    |69 **?**    |1.03        |1           |154 **?**   |

#### For Non Aliased Mazes

|       |Maze4       |Maze5       |MazeA       |MazeF1      |MazeF2      |MazeF3      |Woods1      |Woods14     |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|
|$\phi$ |3.5         |4.61        |4.23        |1.8         |2.5         |3.38        |1.63        |9.5         |
|$\psi$ | **\***     | **\***     | **\***     | **\***     | **\***     | **\***     | **\***     | **\***     |